In [2]:
#平均
import cv2
import os
import numpy as np

imagefilepath = r'/data_hs/sjwlab/meixuewen/project2/dataset/localduo_data/myim_twomany_cro'
fileoutpath=r'/data_hs/sjwlab/meixuewen/project2/dataset/zstack_localduo/avera'
os.makedirs(fileoutpath,exist_ok=True)

def gjjavera(imagefilepath):
    image_files = os.listdir(imagefilepath)
    image_paths = [os.path.join(imagefilepath, f) for f in image_files]

    sum_image = None
    count = 0
    target_width = None
    target_height = None

    for path in image_paths:
        im = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        
        if sum_image is None:
            target_height, target_width = im.shape
            sum_image = np.zeros((target_height, target_width), dtype=np.float32)
        
        if im.shape[0] != target_height or im.shape[1] != target_width:
            im = cv2.resize(im, (target_width, target_height), interpolation=cv2.INTER_AREA)
        
        sum_image += im
        count += 1

    average_image = sum_image / count
    average_image = np.uint8(average_image)
    return average_image


for idfile in os.listdir(filepath):
    id_path=os.path.join(filepath,idfile)
    idout_path=os.path.join(fileoutpath,idfile)
    os.makedirs(idout_path,exist_ok=True)
    for viewfile in os.listdir(id_path):
        view_path=os.path.join(id_path,viewfile)
        viewout=os.path.join(idout_path,viewfile+'.png')
        img=gjjavera(view_path)
        cv2.imwrite(viewout,img)



In [ ]:
import cv2
import os
import numpy as np
from PIL import Image
def is_image_empty(image_path):
    try:
        # 尝试打开图片文件
        with Image.open(image_path) as img:
            # 检查图片的尺寸
            if img.size == (0, 0):
                return True  # 图片尺寸为零，说明是空的
            return False  # 图片尺寸不为零，说明不是空的
    except IOError:
        # 如果图片文件不存在或无法打开，返回True
        return True

n=0

inpath=r'/data_hs/sjwlab/meixuewen/project2/dataset/localduo_data/images'

for id in os.listdir(inpath):
    idpath=os.path.join(inpath,id)
    for im in os.listdir(idpath):
        impath=os.path.join(idpath,im)
        if is_image_empty(impath):
            print(f"图片 {view_path} 是空的。")
            continue
        n+=1
print(n)

In [ ]:




import cv2
import numpy as np
import os

def crop_image_from_right(img, crop_width):
    if img is None:
        raise ValueError("无法读取图像或路径不正确")

    height, width = img.shape[:2]

    if crop_width <= 0 or crop_width > width:
        raise ValueError("裁剪宽度不合理，请选择一个合适的宽度")

    cropped_img = img[:, 0:width - crop_width]

    return cropped_img

def right_black(image):
    if image is None:
        print("Error: 图像无法加载！")
        exit(-1)

    color_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    # 初始化变量用于存储找到黑色像素比例小于90%的列位置
    black_pixel_col = -1

    # 从右到左扫描图像
    for x in range(image.shape[1] - 1, -1, -1):
        # 获取当前列的像素值
        col = image[:, x]
        # 计算当前列黑色像素的数量
        black_pixels_count = np.sum(col == 0)
        # 计算黑色像素的比例
        ratio_black_pixels = black_pixels_count / col.size
        
        # 检查这一列是否有少于90%的黑色像素
        if ratio_black_pixels < 0.9:
            # 如果有，就在这一列画一条红线，并记录列位置
            black_pixel_col = x
            # color_image = cv2.line(color_image, (x, 0), (x, image.shape[0]), (0, 0, 255), 1)
            # 一旦找到一列满足条件，就可以退出循环
            break
    t = image.shape[1] - black_pixel_col - 1
    return t

im=cv2.imread('/data_hs/sjwlab/meixuewen/project2/dataset/orglocal/images_detxt_view_cro/CR2694216LEFT/mlo/DXm.1.2.392.200036.9125.4.0.1326438767.4261224.1573523479.png',0)

# t=right_black(im)
# imcro=crop_image_from_right(im,t)
# cv2.imwrite('righttest.png',imcro)
print(im.shape)

In [ ]:
import os 
import cv2

n=0
imgfile=r'/data_hs/sjwlab/meixuewen/project2/dataset/orglocal/images_detxt_view_cro'
for idfile in os.listdir(imgfile):
    idfilepath = os.path.join(imgfile, idfile)
    for viewfile in os.listdir(idfilepath):
        viewpath = os.path.join(idfilepath, viewfile)
        for im in os.listdir(viewpath):
            if im.endswith('.png'):
                n+=1
print(n)         

In [ ]:
import os.path
import cv2
import pandas as pd
import numpy as np


def split_and_calculate_black_pixels_ratio(img):
    width = img.shape[1]
    split_line = width // 2
    
    left_half = img[:, :split_line]
    right_half = img[:, split_line:]

    def black_pixel_ratio(half_img):
        mask = np.all(half_img == [0, 0, 0], axis=-1)
        black_pixels = np.count_nonzero(mask)
        total_pixels = half_img.size // 3  # 因为是三通道图像
        ratio = black_pixels / total_pixels if total_pixels > 0 else 0
        return ratio
    
    left_ratio = black_pixel_ratio(left_half)
    right_ratio = black_pixel_ratio(right_half)
    
    if left_ratio> right_ratio:
        flag='RIGHT'
    elif left_ratio< right_ratio:
        flag='LEFT'
    return flag

def detect_left_right(s):
    if '左' in s:
        return '左'
    elif '右' in s:
        return '右'
    else:
        return None

# 
df = pd.read_excel(r'/data_hs/sjwlab/meixuewen/project3/datadeal/test/test_.xlsx')

for index, row in df.iterrows():
    name = row['影像号']
    leftright=row['石蜡病理诊断']
    le=row['良/恶 ']
    # print(name,detect_left_right(leftright))
    view=detect_left_right(leftright)
    if view=='左':
        flag='LEFT'
    elif view=='右':
        flag=='RIGHT'
    if le=='恶':
        label=1
    elif le=='良':
        label=0
    print(flag)


    

In [17]:
from pathlib import Path

p = Path("/home/user/documents")
home = Path.home()
print(home)
new_path = p /'example.txt'
# print(new_path)
# print(new_path.is_file()) 
# print(new_path.is_dir()) 
new_path.name
new_path.suffix
new_path.stem


/home/meixuewen


'example'